# 创建 MCP 工具集

AgentKit 网关支持创建 MCP 工具集。MCP 工具集是 MCP 工具的集合，当你在 MCP 工具集中添加 MCP 服务时，MCP 工具集将自动索引 MCP 服务中的 MCP 工具。MCP 工具集也是 1 个 MCP 服务，在调用 MCP 工具集时，MCP 工具集会使用语义搜索，通过对调用意图和 MCP 工具描述进行语义匹配，并返回匹配程度最高的工具。使用 MCP 工具集有助于提高工具匹配准确度和准确性，并且降低 Token 的消耗。

在本教程中，我们将演示如何创建 MCP 工具集，你将学会，

1. 如何在 AgentKit 网关中创建 MCP 工具集，
2. 如何使用 MCP 工具集。

在开始本教程之前，请确保你已经**正确配置了凭证**。

> **注意**：如果你还没有配置凭证，请先学习 [`01-tutorials/01-runtime/00_configuration_and_credentials.ipynb`](../01-runtime/00_configuration_and_credentials.ipynb) 教程完成配置。

## 1. 创建 MCP 工具集

AgentKit 网关支持创建 MCP 工具集。MCP 工具集是 1 个 MCP 服务，通过 MCP 协议提供服务，你可以将 MCP 工具集的服务地址配置在智能体中使用 MCP 工具集。对于配置在 MCP 工具集内的 MCP 服务，MCP 工具集会在 MCP 工具发生变更时索引，并且将 MCP 工具的元数据保存在 VikingDB 向量数据库中。在智能体调用 MCP 工具集时，智能体首先会生成任务描述并调用 `mcp_search_tool` 工具，MCP 工具集将进行语义匹配并返回匹配程度最高的 10 个工具。然后，智能体根据返回的 MCP 工具元数据调用 `mcp_use_tool` 工具，MCP 工具集将完成最终的调用并返回调用结果。

MCP 工具集有助于提高工具匹配准备度和准确性，并且降低 Token 的消耗。智能体在启动时会列出所有的 MCP 工具，并且在和大语言模型的第 1 轮会话前注入。一个典型的 MCP 服务器包含数十个 MCP 工具，GitHub 的 MCP 服务器包含将近 100 个工具，而飞书的 MCP 服务器更是包含了 700 余个工具。当你接入 50 个 MCP 工具时，大语言模型的上下文将被 MCP 工具的元数据占用，这不仅会影响大语言模型对 MCP 工具调用的决策，还会大幅度增加 Token 的消耗。使用 MCP 工具集可以提升 MCP 工具调用的准确性，减少大语言模型的响应时间，并且将 Token 的消耗量降低到十分之一及以下。

在创建 MCP 工具集之前，我们首先创建 1 个 MCP 服务。我们使用 `@modelcontextprotocol/server-everything` 公共包创建 MCP 服务。`@modelcontextprotocol/server-everything` 包含了大量的 MCP 工具，是我们测试 MCP 工具集的不二之选。

In [ ]:
from agentkit.sdk.mcp.client import AgentkitMCPClient
from agentkit.sdk.mcp.types import (
    CreateMCPServiceRequest,
    NetworkForCreateMCPService,
    InboundAuthorizerForCreateMCPService,
    InboundAuthorizerAuthorizerForCreateMCPService,
    InboundAuthorizerAuthorizerKeyAuthForCreateMCPService,
    InboundAuthorizerAuthorizerKeyAuthApiKeysItemForCreateMCPService,
    BackendForCreateMCPService,
    BackendCustomMcpForCreateMCPService,
    BackendCustomMcpPublicPackageForCreateMCPService,
)
import json
import uuid

id = str(uuid.uuid4())[-8:]
mcp_config = {
    "mcpServers": {
        "sequential-thinking": {
            "command": "npx",
            "args": ["-y", "@modelcontextprotocol/server-everything"],
        },
    },
}

client = AgentkitMCPClient()
request = CreateMCPServiceRequest(
    Name=f"mcp-service-{id}",
    ProtocolType="MCP",
    Path="/mcp",
    NetworkConfiguration=NetworkForCreateMCPService(
        EnablePublicNetwork=True,
        EnablePrivateNetwork=False,
    ),
    InboundAuthorizerConfiguration=InboundAuthorizerForCreateMCPService(
        AuthorizerType="ApiKey",
        Authorizer=InboundAuthorizerAuthorizerForCreateMCPService(
            KeyAuth=InboundAuthorizerAuthorizerKeyAuthForCreateMCPService(
                ApiKeys=[
                    InboundAuthorizerAuthorizerKeyAuthApiKeysItemForCreateMCPService(
                        Name=f"mcp-service-api-key-{id}",
                    ),
                ],
            ),
        ),
    ),
    BackendType="CustomPublic",
    BackendConfiguration=BackendForCreateMCPService(
        CustomMcpConfiguration=BackendCustomMcpForCreateMCPService(
            PublicPackage=BackendCustomMcpPublicPackageForCreateMCPService(
                McpType="Remote",
                PackageManagerType="NPX",
                RawConfig=json.dumps(mcp_config),
            )
        ),
    ),
    ProjectName="default",
)

mcp_service_id = ""
try:
    response = client.create_mcp_service(request)
    mcp_service_id = response.mcp_service_id
    print(f"MCP service created successfully! MCP service ID: {mcp_service_id}")
except Exception as e:
    print(f"An error occurred: {e}")
    raise e

In [ ]:
import time
from agentkit.sdk.mcp.types import (
    GetMCPServiceRequest,
)

request = GetMCPServiceRequest(
    MCPServiceId=mcp_service_id,
)

endpoint = ""
api_key = ""
try:
    while True:
        response = client.get_mcp_service(request)
        if response.mcp_service.status == "Ready":
            endpoint = response.mcp_service.network_configurations[0].endpoint
            api_key = response.mcp_service.inbound_authorizer_configuration.authorizer.key_auth.api_keys[0].key
            break
        if response.mcp_service.status != "Creating":
            print(f"The MCP service is in {response.mcp_service.status}")
            raise RuntimeError()
        time.sleep(2)
except Exception as e:
    print(f"An error occurred: {e}")
    raise e

url = f"{endpoint}/mcp"
print(f"MCP service URL: {url}")
print(
    f'MCP service API key: {api_key}\n  (Hint: Add a "Authorization: Bearer {api_key}" request header to access the MCP service)'
)

很好，我们已经创建了 `@modelcontextprotocol/server-everything` MCP 服务，让我们来看一下这个 MCP 服务中包含了多少工具。

In [ ]:
!pip install fastmcp --quiet

In [ ]:
from fastmcp import Client
from fastmcp.client.transports import StreamableHttpTransport

transport = StreamableHttpTransport(
    url=url,
    headers={
        "Authorization": f"Bearer {api_key}",
    }
)
mcp_client = Client(transport)

async with mcp_client:
    tools = await mcp_client.list_tools()
    print(f"The MCP server has {len(tools)} MCP tools.")
    for tool in tools:
        print(f"- {tool.name}")

`@modelcontextprotocol/server-everything` MCP 服务包含了各种各样的样例工具。这些工具种类繁多，从加法计算的 `add` 到压缩文件的 `zip`。而有的工具名称之间又存在一定的混淆，例如 `getResourceReference` 和 `getResourceLinks`。在这种多个 MCP 工具的复合场景下，使用 MCP 工具集不妨为一个好主意。

接下来，我们将为刚刚创建的 MCP 服务创建 MCP 工具集。

In [ ]:
from agentkit.sdk.mcp.types import (
    CreateMCPToolsetRequest,
    NetworkForCreateMCPToolset,
    AuthorizerForCreateMCPToolset,
    AuthorizerAuthorizerForCreateMCPToolset,
    AuthorizerAuthorizerKeyAuthForCreateMCPToolset,
    AuthorizerAuthorizerKeyAuthApiKeysItemForCreateMCPToolset,
)

id_2 = str(uuid.uuid4())[-8:]

request = CreateMCPToolsetRequest(
    Name=f"mcp-toolset-{id_2}",
    Path="/mcp",
    NetworkConfiguration=NetworkForCreateMCPToolset(
        EnablePublicNetwork=True,
        EnablePrivateNetwork=False,
    ),
    AuthorizerConfiguration=AuthorizerForCreateMCPToolset(
        AuthorizerType="ApiKey",
        Authorizer=AuthorizerAuthorizerForCreateMCPToolset(
            KeyAuth=AuthorizerAuthorizerKeyAuthForCreateMCPToolset(
                ApiKeys=[
                    AuthorizerAuthorizerKeyAuthApiKeysItemForCreateMCPToolset(
                        Name=f"mcp-toolset-api-key-{id_2}",
                    ),
                ],
            ),
        ),
    ),
    # AgentKit 网关 MCP 服务 ID，可以选择多个 MCP 服务。
    MCPServiceIds=[
        mcp_service_id,
    ],
    ProjectName="default",
)

mcp_toolset_id = ""
try:
    response = client.create_mcp_toolset(request)
    mcp_toolset_id = response.mcp_toolset_id
    print(f"MCP toolset created successfully! MCP toolset ID: {mcp_toolset_id}")
except Exception as e:
    print(f"An error occurred: {e}")
    raise e

AgentKit Gateway 部署 MCP 工具集需要一段时间，MCP 工具集索引 MCP 工具也需要一定的时间。等到 MCP 工具集部署完成后，我们尝试调用 `mcp_search_tool` 和 `mcp_use_tool` 来看看效果如何。

In [ ]:
import time
from agentkit.sdk.mcp.types import (
    GetMCPToolsetRequest,
)

request = GetMCPToolsetRequest(
    MCPToolsetId=mcp_toolset_id,
)

endpoint_2 = ""
api_key_2 = ""
try:
    while True:
        response = client.get_mcp_toolset(request)
        if response.mcp_toolset.status == "Ready":
            endpoint_2 = response.mcp_toolset.network_configurations[0].endpoint
            api_key_2 = response.mcp_toolset.authorizer_configuration.authorizer.key_auth.api_keys[0].key
            break
        if response.mcp_toolset.status != "Creating":
            print(f"The MCP toolset is in {response.mcp_toolset.status}")
            raise RuntimeError()
        time.sleep(2)
except Exception as e:
    print(f"An error occurred: {e}")
    raise e

url_2 = f"{endpoint_2}/mcp"
print(f"MCP toolset URL: {url_2}")
print(
    f'MCP toolset API key: {api_key_2}\n  (Hint: Add a "Authorization: Bearer {api_key_2}" request header to access the MCP toolset)'
)

In [ ]:
from mcp.types import TextContent

transport = StreamableHttpTransport(
    url=url_2,
    headers={
        "Authorization": f"Bearer {api_key_2}",
    },
)
mcp_client = Client(transport)

async with mcp_client:
    result = await mcp_client.call_tool(
        "mcp_search_tool",
        {
            "task_description": [
                "Add 2 + 3",
            ],
        },
    )
    text: TextContent = result.content[0]
    metadata = text.text.splitlines()[1]
    tool = json.loads(metadata)[0]
    print(f'The MCP toolset finds the most specific tool is "{tool["tool_name"]}".')

    result = await mcp_client.call_tool(
        "mcp_use_tool",
        {
            "use_tool_info": [
                {
                    "server_id": tool["server_id"],
                    "server_name": tool["server_name"],
                    "tool_id": tool["tool_id"],
                    "tool_name": tool["tool_name"],
                    "argument": {
                        "a": 2,
                        "b": 3,
                    },
                },
            ],
        },
    )
    text: TextContent = result.content[0]
    tool_text = json.loads(text.text)[0]["result"][0]["text"]
    print(f"The MCP toolset call tool result is \"{tool_text}\".")

恭喜！你已经使用 AgentKit 网关部署了 MCP 工具集，这个 MCP 工具集指向 `@modelcontextprotocol/server-everything` MCP 服务，并且可以通过语义搜索选出最适合当前任务的 MCP 工具集。

## 总结

恭喜你！现在你已经掌握了创建 AgentKit 网关 MCP 工具集的方法，

1. 创建 MCP 工具集。

现在，请继续学习其他教程。